In [1]:
import train
import numpy as np
import api.capital_features_api as cf
import pandas as pd
import mplfinance as mpf
from datetime import datetime
import joblib
import matplotlib.pyplot as plt
import model_util as mu
from tensorflow import keras
import os
import shutil

model_name = 'v12.80.200.i5.all.lstm1.sp22y1d.20240210.2'
model_dir = './model/' + model_name + '/'
os.makedirs(model_dir, exist_ok=True)
scaler_save_file = model_dir + 'scaler.save'
scaler_json_file = model_dir + 'scaler.json'
model_h5_file = model_dir + 'model.h5'
model_tf_dir_file = model_dir + 'model_tf'
model_tf_dir_scaler_json_file = model_tf_dir_file + '/scaler.json'

In [2]:
train_ud_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=2059175)
validation_ud_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=2059176)
Y_ud_train = train_ud_data['upSignals']
Y_ud_val = validation_ud_data['upSignals']

In [3]:
train_ud_dataset, validation_ud_dataset, scaler_ud = train.to_dataset(train_ud_data['features'], validation_ud_data['features'], Y_ud_train, Y_ud_val)
weights_ud = train.to_weights(Y_ud_train)
joblib.dump(scaler_ud, scaler_save_file)
train.save_scaler(scaler_ud, scaler_json_file)

In [4]:
model_ud, callbacks_ud = train.get_model(80, 5, model_name)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 80, 128)           68608     
                                                                 
 dropout (Dropout)           (None, 80, 128)           0         
                                                                 
 batch_normalization (BatchN  (None, 80, 128)          512       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                        

In [ ]:
his_ud = model_ud.fit(train_ud_dataset, epochs=500, validation_data=validation_ud_dataset, verbose=1, callbacks=callbacks_ud, class_weight=weights_ud)
model_ud.save(model_h5_file)
model_ud.save(model_tf_dir_file, save_format='tf')
shutil.copy2(scaler_json_file, model_tf_dir_scaler_json_file)

Epoch 1/500
8573/8574 [============================>.] - ETA: 0s - loss: 0.7035 - accuracy: 0.5265 - precision: 0.5228 - recall: 0.6103
Epoch 00001: val_loss improved from inf to 0.69565, saving model to ./checkpoint/v12.80.200.i5.all.lstm1.sp22y1d.20240210.2/epoch_01.ckpt
8574/8574 [==============================] - 238s 27ms/step - loss: 0.7035 - accuracy: 0.5265 - precision: 0.5228 - recall: 0.6103 - val_loss: 0.6956 - val_accuracy: 0.5224 - val_precision: 0.5270 - val_recall: 0.4380 - lr: 0.0010
Epoch 2/500
8573/8574 [============================>.] - ETA: 0s - loss: 0.6928 - accuracy: 0.5294 - precision: 0.5239 - recall: 0.6448
Epoch 00002: val_loss improved from 0.69565 to 0.69542, saving model to ./checkpoint/v12.80.200.i5.all.lstm1.sp22y1d.20240210.2/epoch_02.ckpt
8574/8574 [==============================] - 237s 28ms/step - loss: 0.6928 - accuracy: 0.5294 - precision: 0.5239 - recall: 0.6448 - val_loss: 0.6954 - val_accuracy: 0.5075 - val_precision: 0.5355 - val_recall: 0.1133

In [ ]:

plt.plot(his_ud.history['accuracy'], label='accuracy')
plt.plot(his_ud.history['val_accuracy'], label='val_accuracy')
plt.plot(his_ud.history['loss'], label='loss')
plt.plot(his_ud.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
model_ud = keras.models.load_model(model_tf_dir_file)
scaler_ud = joblib.load(scaler_save_file)

In [ ]:
prediction_data = cf.get_prediction_data(cf.docker_prediction_data_url, train_config_id=2059175, start="2017-07-11", end=datetime.now().strftime("%Y-%m-%d"), code="QQQ")
kline = cf.get_kline_histroy(cf.docker_kline_history_url, start="2017-07-11", end=datetime.now().strftime("%Y-%m-%d"), code='QQQ')
df = cf.his_to_df(kline)

In [ ]:
X_prediction_scaled = train.to_prediction_scaled(prediction_data, scaler_ud)
Y_ud_prediction = model_ud.predict(X_prediction_scaled)

In [ ]:
predictions_ud_series = pd.Series(Y_ud_prediction.flatten(), index=df.index)
buy_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
buy_signals[predictions_ud_series > 0.7] = df['low'][predictions_ud_series > 0.7] * 0.99
buy_markers = mpf.make_addplot(buy_signals, type='scatter', markersize=50, marker='^', color='g')

sell_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
sell_signals[predictions_ud_series < 0.2] = df['high'][predictions_ud_series < 0.2] * 1.01
sell_signals = mpf.make_addplot(sell_signals, type='scatter', markersize=50, marker='_', color='gray')

mpf.plot(df, type='candle', style='charles', addplot=[buy_markers, sell_signals], volume=True, figsize=(24, 16))